In [1]:
import sys
sys.path.append('../../build')
import cv2 as cv
import numpy as np
import libry as ry
import time
print(cv.__version__)

**ry-c++-log** /home/tim/git/robotics-course_tim/rai/rai/ry/ry.cpp:init_LogToPythonConsole:34(0) initializing ry log callback** INFO:/home/tim/git/robotics-course_tim/rai/rai/Core/util.cpp:initCmdLine:602(1) ** cmd line arguments: 'rai-pybind -python'

** INFO:/home/tim/git/robotics-course_tim/rai/rai/Core/util.cpp:initCmdLine:606(1) ** run path: '/home/tim/git/robotics-course_tim/course3-Simulation/04-opencv'
** INFO:/home/tim/git/robotics-course_tim/rai/rai/Core/graph.cpp:initParameters:1365(1) ** parsed parameters:
{python}


** INFO:/home/tim/git/robotics-course_tim/rai/rai/ry/ry.cpp:init_LogToPythonConsole:34(0) initializing ry log callback
4.5.3


In [2]:
#-- Add REAL WORLD configuration and camera
RealWorld = ry.Config()
RealWorld.addFile("../../scenarios/ex03_redball.g")
S = RealWorld.simulation(ry.SimulatorEngine.bullet, True)
S.addSensor("camera")

C = ry.Config()
C.addFile('../../scenarios/pandasTable.g')
D = C.view()
cameraFrame = C.frame("camera")

In [3]:
#the focal length
f = 0.895
f = f * 360.
#the relative pose of the camera
# pcl.setRelativePose('d(-90 0 0 1) t(-.08 .205 .115) d(26 1 0 0) d(-1 0 1 0) d(6 0 0 1) ')
fxfypxpy = [f, f, 320., 180.]

In [4]:
points = []
tau = .01

for t in range(300):
    time.sleep(0.01)

    #grab sensor readings from the simulation
    q = S.get_q()
    if t%10 == 0:
        [rgb, depth] = S.getImageAndDepth()  #we don't need images with 100Hz, rendering is slow
        points = S.depthData2pointCloud(depth, fxfypxpy)
        cameraFrame.setPointCloud(points, rgb)
            
        if len(rgb)>0: cv.imshow('OPENCV - rgb', cv.cvtColor(rgb, cv.COLOR_BGR2RGB))
        if len(depth)>0: cv.imshow('OPENCV - depth', 0.5* depth)

        if cv.waitKey(1) & 0xFF == ord('q'):
            break
        
    S.step([], tau, ry.ControlMode.none)




In [5]:
rgb = cv.cvtColor(rgb, cv.COLOR_BGR2RGB)
hsv = cv.cvtColor(rgb, cv.COLOR_BGR2HSV)
mask1 = cv.inRange(hsv, (0, 120, 70), (10, 255, 255))
mask2 = cv.inRange(hsv, (170, 120, 70), (180, 255, 255))
#if len(mask1)>0: cv.imshow('OPENCV - mask1',  mask1)
#if len(mask2)>0: cv.imshow('OPENCV - mask2',  mask2)
mask = mask1 + mask2
# create empty mask
filtered_mask = np.zeros(mask.shape, np.uint8)
#if len(filtered_mask)>0: cv.imshow('OPENCV - filtered_mask',  filtered_mask)
# find contours
contours, _ = cv.findContours(mask, cv.RETR_LIST, cv.CHAIN_APPROX_SIMPLE)
if len(contours) > 0:
    # find largest contour
    largest, idx = 0., None
    for i, c in enumerate(contours):
        # remove noise
        if c.shape[0] < 10: continue
        if cv.contourArea(c) > largest:
            largest = cv.contourArea(c)
            idx = i
    cv.drawContours(filtered_mask, contours[idx], -1, (255, 255, 255), -1)


In [6]:
    # if len(mask) > 0: cv.imshow('OPENCV - mask1', mask * 255)
    # cv.waitKey(100)
    mask_pixels = np.where(filtered_mask > 0)
    pointcloud = np.empty((mask_pixels[0].shape[0], 3))
    pointcloud[:, 0] = mask_pixels[1]  # x pixels
    pointcloud[:, 1] = mask_pixels[0]  # y pixels
    pointcloud[:, 2] = depth[mask_pixels[0], mask_pixels[1]]


In [7]:
    points = np.empty(np.shape(pointcloud))
    for i, p in enumerate(pointcloud):
        x = p[0]
        y = p[1]
        d = p[2]

        px = fxfypxpy[-2]
        py = fxfypxpy[-1]

        x_ = d * (x - px) / fxfypxpy[0]
        y_ = -d * (y - py) / fxfypxpy[1]
        z_ = -d
        points[i] = [x_, y_, z_]

In [8]:
position = points.mean(axis=0)

In [9]:
mk_obj1 = C.addFrame("mk_obj1","camera")
mk_obj1.setShape(ry.ST.marker,[.5])

mk_obj1.setRelativePosition(position)


In [10]:
cv.destroyAllWindows()

In [ ]:
import os
os._exit(0)